In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../raw-files/candidates.csv')

In [17]:
df['name'] = df.apply(lambda r: '%s %s' % (r['FirstName'], r['LastName']), axis=1)
df['desc'] = df.apply(lambda r: 'สส.เขต (%s เขต %s)' % (r['province_name'], r['zone_number']), axis=1)

In [4]:
df_party = df[df.PartyName == 'พลังประชารัฐ']

In [22]:
def extract_party_politician(party):
    df_party = df[df.PartyName == party]
    df_party = df_party[['name', 'desc']]
    df_party.to_csv('../data/party-politicians/%s.csv' % party, index=False, header=True)
    return df_party

In [23]:
for p in list(set(df.PartyName.values)):
    extract_party_politician(p)

In [31]:
df[df.FirstName == "ณิชชา"]

,Unnamed: 0,CandidateId,CandidateNo,FirstName,LastName,PartyName,PartyNumber,Title,Votable,province_id,province_name,zone_number
1609,1609,be3ba912-36b0-4086-a40f-9a74d672617a,12,ณิชชา,แก้วนุ้ย,ชาติพัฒนา,45,นางสาว,True,93,พัทลุง,2
1764,1764,c6774cd0-8835-4709-abdc-19630c17a00e,8,ณิชชา,พิมพ์สว่าง,พลังท้องถิ่นไท,124,นาง,True,76,เพชรบุรี,1
10635,10635,83142202-4b16-400d-9085-42f340152445,10,ณิชชา,บุญลือ,อนาคตใหม่,177,นางสาว,True,10,กรุงเทพมหานคร,13


In [9]:
list(set(df.PartyName.values))

['รวมพลังประชาชาติไทย',
 'ภูมิพลังเกษตรกรไทย',
 'ยางพาราไทย',
 'ประชาธิปัตย์',
 'ชาติไทยพัฒนา',
 'พลังไทยรักไทย',
 'พลเมืองไทย',
 'คนงานไทย',
 'ประชาธรรมไทย',
 'พลังประชาธิปไตย',
 'เครือข่ายชาวนาแห่งประเทศไทย',
 'เพื่อธรรม',
 'ประชาธิปไตยเพื่อประชาชน',
 'พลังท้องถิ่นไท',
 'เพื่อชาติ',
 'กรีน',
 'เสรีรวมไทย',
 'พลังแผ่นดินทอง',
 'พลังประชารัฐ',
 'ไทยรุ่งเรือง',
 'ความหวังใหม่',
 'แทนคุณแผ่นดิน',
 'สามัญชน ',
 'เพื่อไทย',
 'ฐานรากไทย',
 'เพื่อแผ่นดิน',
 'กสิกรไทย',
 'เพื่อไทยพัฒนา',
 'พลังแรงงานไทย',
 'พลังชาติไทย',
 'พลังไทยรักชาติ',
 'ภาคีเครือข่ายไทย',
 'สยามพัฒนา',
 'ไทรักธรรม',
 'ประชาไทย',
 'พัฒนาประเทศไทย',
 'ประชานิยม',
 'ชาติพันธุ์ไทย',
 'พลังสหกรณ์',
 'ประชาชาติ',
 'อนาคตใหม่',
 'พลังไทยดี',
 'พลังสังคม',
 'ไทยศรีวิไลย์',
 'รักษ์ธรรม',
 'ประชากรไทย',
 'ประชาธิปไตยใหม่',
 'สังคมประชาธิปไตยไทย',
 'คนธรรมดาแห่งประเทศไทย',
 'พลังคนกีฬา',
 'พลังครูไทย',
 'มติประชา',
 'ภูมิใจไทย',
 'คลองไทย',
 'พลังรัก',
 'ประชาภิวัฒน์',
 'พลังศรัทธา',
 'ทางเลือกใหม่',
 'รักษ์ผืนป่าประเทศไทย',
 'อนาค